In [2]:
import torch 
import pickle
import io
import pandas as pd
import nltk

from modules.models import SummaryModel, ModelConfig
from modules.datasets import SentenceClusterDataset, DocSplitDataset
from transformers import BertTokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
from rouge import Rouge

In [3]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /Users/d059221/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /Users/d059221/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /Users/d059221/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /Users/d059221/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /Users/d059221/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /Users/d059221/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nl

True

## Sample Data

In [31]:
tokenizer = BertTokenizer.from_pretrained('bert-base-german-cased')

data = pd.read_csv('./data//export-articles-de-2021-09-08T14:45:27.515899+00:00')
data = data[['CONTENT']].dropna()

#df_test = data.iloc[50000:50100]
df_test = data.iloc[0:20]
df_test['sentences'] = df_test['CONTENT'].apply(
    lambda x: nltk.sent_tokenize(
        x, language='german'))
df_test['sentences'] = df_test['sentences'].apply(
    lambda x: [
        s for s in x if len(
            s.split(' ')) > 4 and len(
            s.split(' ')) < 32])

/Users/d059221/opt/anaconda3/envs/master-thesis-colab/lib/python3.7/site-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
/Users/d059221/opt/anaconda3/envs/master-thesis-colab/lib/python3.7/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/Users/d059221/opt/anaconda3/envs/master-thesis-colab/lib/pytho

## Sentence Cluster

In [11]:
checkpoint = torch.load('./checkpoints/mobbs/sent-cluster/checkpoint-epoch-9', map_location=torch.device('cpu'))

In [12]:
encoder_config = ModelConfig(checkpoint['encoder_config'])
decoder_config = ModelConfig(checkpoint['decoder_config'])

model = SummaryModel(encoder_config, decoder_config)
model.load_state_dict(checkpoint['model_state_dict'])
model.eval()

/Users/d059221/opt/anaconda3/envs/master-thesis-colab/lib/python3.7/site-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model traine

SummaryModel(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine

## Select Sample

In [32]:
sbert = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
i_example = 0

example_sentences = df_test['sentences'].iloc[i_example]
sent_emb = sbert.encode(example_sentences)

num_sentences = len(example_sentences)
num_clusters = int(num_sentences * 0.3)
num_clusters = 1 if num_clusters == 0 else num_clusters
clustering_model = KMeans(n_clusters=num_clusters)
clustering_model.fit(sent_emb)
cluster_assignment = clustering_model.labels_

clustered_sentences = [[] for i in range(num_clusters)]
for sentence_id, cluster_id in enumerate(cluster_assignment):
    clustered_sentences[cluster_id].append(example_sentences[sentence_id])

/Users/d059221/opt/anaconda3/envs/master-thesis-colab/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")


In [43]:
clustered_sentences

[['US-Präsident Joe Biden hat sich erneut gegen die deutsch-russische Erdgas-Pipeline Nord Stream 2 ausgesprochen.',
  '„Ich bin seit Langem gegen Nord Stream 2“, sagte Biden am Donnerstag im Weißen Haus.',
  'Andererseits gebe es Druck aus dem Kongress, Sanktionen zu verhängen.Die Biden-Regierung hat neue Strafmaßnahmen gegen Nord Stream 2 bislang bloß angedroht, aber nicht umgesetzt.'],
 ['Bislang haben die USA wegen Nord Stream 2 lediglich Sanktionen gegen die russische Betreiberfirma eines Verlegeschiffes verhängt – diese Strafmaßnahmen hatte im Januar noch die Regierung des Republikaners Trump verkündet.',
  'Die USA befürchten eine zu starke Abhängigkeit Europas von russischem Gas und wollen das Projekt mit Sanktionen stoppen.',
  '„Die USA sind nicht darauf aus, einen Kreislauf der Eskalation und des Konflikts mit Russland einzuleiten“, sagte Biden am Donnerstag im Weißen Haus.',
  'Unter den Diplomaten der Vertretung in Washington seien auch fünf Mitarbeiter russischer Geheimdi

In [33]:
print('Sample Text')
print(df_test['CONTENT'].iloc[i_example])

Sample Text
US-Präsident Joe Biden hat sich erneut gegen die deutsch-russische Erdgas-Pipeline Nord Stream 2 ausgesprochen. „Ich bin seit Langem gegen Nord Stream 2“, sagte Biden am Donnerstag im Weißen Haus. Auf die Frage, warum das Projekt nicht von den jüngsten US-Sanktionen gegen Russland betroffen ist, entgegnete der Demokrat: „Nord Stream 2 ist ein kompliziertes Thema, das unsere Verbündeten in Europa betrifft.“ Das Thema sei aber weiter auf dem Tisch. Besonders die Republikaner im Kongress drängen Biden dazu, weitere Sanktionen gegen Nord Stream 2 zu verhängen, um die Fertigstellung der Ostsee-Pipeline noch zu verhindern.Die Nachrichtenseite „Politico“ berichtete, das US-Justizministerium habe im vergangenen Monat grünes Licht für Sanktionen gegen die Nord Stream 2 AG gegeben. Nach regierungsinternen Diskussionen sei das aber revidiert worden. Ein hochrangiger Regierungsvertreter sagte nach Angaben von „Politico“, man befinde sich in einer schwierigen Lage. Einerseits wolle man 

In [34]:
summary = ''
summ_sentences = list()
for cluster in clustered_sentences:
    tok = tokenizer(cluster, padding=True, truncation=True, add_special_tokens=True, return_tensors="pt")
    tok_tensor = torch.tensor(tok['input_ids'].numpy()[:,1:])
    summ_sentence = model.summarize_sequences({'input_ids': tok_tensor})
    summ_sentence = tokenizer.decode(summ_sentence[0][0], skip_special_tokens=True)
    print(summ_sentence)
    summ_sentences.append(summ_sentence)

summary = ' '.join(summ_sentences)

Ich hat Bidens Präsident damals - Sanktionen gegen Nord Stream 2, aber es einen testen. Die Corona in Großbritannien dafür.
Die USA sagte die Präsident Donald Trump, die umstrittene Staaten sind den aus ( bundesweit statt russische.
Wie regnet hat aber auch der Kremen in Arzt hervor.
Er sagte Präsident Donald Baerbock, dabei nicht wie.
Am Donnerstag sagte der US - Präsident, nach seinem Vorschlag solle ein solches Treffen im Sommer in Europa stattfinden.


In [19]:
rouge = Rouge()
rouge_2 = list()
for a in range(100):
    example_sentences = df_test['sentences'].iloc[a]
    sent_emb = sbert.encode(example_sentences)

    num_sentences = len(example_sentences)
    num_clusters = int(num_sentences * 0.3)
    num_clusters = 1 if num_clusters == 0 else num_clusters
    clustering_model = KMeans(n_clusters=num_clusters)
    clustering_model.fit(sent_emb)
    cluster_assignment = clustering_model.labels_

    clustered_sentences = [[] for i in range(num_clusters)]
    for sentence_id, cluster_id in enumerate(cluster_assignment):
        clustered_sentences[cluster_id].append(example_sentences[sentence_id])

    summary = ''
    summ_sentences = list()
    for cluster in clustered_sentences:
        tok = tokenizer(cluster, padding=True, truncation=True, add_special_tokens=True, return_tensors="pt")
        tok_tensor = torch.tensor(tok['input_ids'].numpy()[:,1:])
        summ_sentence = model.summarize_sequences({'input_ids': tok_tensor})
        summ_sentence = tokenizer.decode(summ_sentence[0][0], skip_special_tokens=True)
        #print(summ_sentence)
        summ_sentences.append(summ_sentence)

    summary = ' '.join(summ_sentences)

    example_article = df_test['CONTENT'].iloc[a]
    scores = rouge.get_scores(summary, example_article)
    rouge_2.append(scores[0]['rouge-2']['f'])

/Users/d059221/opt/anaconda3/envs/master-thesis-colab/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")
/Users/d059221/opt/anaconda3/envs/master-thesis-colab/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")
/Users/d059221/opt/anaconda3/envs/master-thesis-colab/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `padding`=`True`.")
/Users/d059221/opt/anaconda3/envs/master-thesis-colab/lib/python3.7/site-packages/transformers/tokenization_utils_base.py:2227: UserWarning: `max_length` is ignored when `padding`=`True`.
  warnings.warn("`max_length` is ignored when `pad

In [27]:
rouge_2.index(0.06080347284429783)

16

Get the METEOR score

## Use sub-doc approach

In [35]:
checkpoint = torch.load('./checkpoints/mobbs/doc-split/checkpoint-epoch-8', map_location=torch.device('cpu'))

encoder_config = checkpoint['encoder_config']
decoder_config = checkpoint['decoder_config']

model = SummaryModel(ModelConfig(encoder_config), ModelConfig(decoder_config))
model.load_state_dict(checkpoint['model_state_dict'])

model.eval()
print('')

/Users/d059221/opt/anaconda3/envs/master-thesis-colab/lib/python3.7/site-packages/transformers/configuration_utils.py:337: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  "Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 "
Some weights of the model checkpoint at bert-base-german-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model traine

In [36]:
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

with open('./data/doc-split/train_set-2.pt', mode='rb') as f:
  train_set = CPU_Unpickler(f).load()

with open('./data/doc-split/test_set-2.pt', mode='rb') as f:
  test_set = CPU_Unpickler(f).load()

In [39]:
i_example = 0
sample = test_set.dataset.df.iloc[i_example]['enc_tokens']
summary = model.summarize_sequences(sample)
summary = tokenizer.decode(summary[0][0], skip_special_tokens=True)
print(summary)

Am Donnerstag verkündete Biden der USA sich an die US - Regierung und Biden. Biden sagte am Montag, er wolle nicht mit dem russischen Präsidenten Wladimir Putin sprechen. Die US - Außenminister Antony Blinken und die US - Außenminister Antony Blinken und die US - Außenminister Antony Blinken und US - Außenminister Antony Blinken wollen nach Angaben von US - Außenminister Antony Blinken in einem Telefonat mit dem russischen Präsidenten Wladimir Putin auf die Frage nach möglichen Sanktionen gegen den Kreml - Kritiker Alexej Nawalny reagiert werden. Ich bin sehr besorgt, sagte Biden


In [122]:
example_article = df_test['CONTENT'].iloc[i_example]
rouge.get_scores(summary, example_article)

[{'rouge-1': {'r': 0.10631229235880399,
   'p': 0.6037735849056604,
   'f': 0.1807909579059338},
  'rouge-2': {'r': 0.018867924528301886,
   'p': 0.13846153846153847,
   'f': 0.03321032999244305},
  'rouge-l': {'r': 0.10299003322259136,
   'p': 0.5849056603773585,
   'f': 0.17514124039180953}}]

In [109]:
example_article

'Bei neuen gewaltsamen Auseinandersetzungen zwischen Palästinensern und israelischen Polizisten in Jerusalem sind am Samstagabend nach Angaben von Rettungskräften mehr als 50 Menschen verletzt worden. „Wir zählen bislang 53 Verletzte“, sagte ein Sprecher des palästinensischen Roten Halbmonds der Nachrichtenagentur AFP.\xa0\nZu den neuen Zusammenstößen kam es in der Altstadt von Jerusalem und im Ost-Jerusalemer Stadtteil Scheich Dscharrah. Dort gingen erneut Palästinenser auf die Straße, um gegen drohende Zwangsräumungen zu protestieren, und warfen mit Steinen in Richtung israelischer Sicherheitskräfte. Diese setzten Wasserwerfer ein und versuchten, die Demonstranten mit übelriechendem Wasser zu vertreiben, wie ein AFP-Korrespondent berichtete.\nJugendliche zündeten laut Nachrichtenagentur Reuters zudem Feuer an und zerstörten Barrikaden, die den Zugang zur Altstadt versperren sollten.\nBereits am Abend zuvor hatte es Zusammenstöße auf dem Tempelberg und in anderen Teilen Ost-Jerusalems

In [40]:
d = data.iloc[50000:50010]
split_val = DocSplitDataset(tokenizer, d)

/Users/d059221/Workspace/Git/master-thesis/modules/datasets.py:196: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lambda x: nltk.sent_tokenize(
/Users/d059221/Workspace/Git/master-thesis/modules/datasets.py:198: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['content_length'] = df['CONTENT'].str.len()
Keyword arguments {'max_lenght': 510} not recognized.
Keyword arguments {'max_lenght': 510} not recognized.
Keyword arguments {'max_lenght': 510} not recognized.
Keyword arguments {'max_lenght': 510} not rec

In [41]:
sample = split_val.X_enc[0]
summary = model.summarize_sequences(sample)
summary = tokenizer.decode(summary[0][0], skip_special_tokens=True)
print(summary)

Der wie Krankenschwester und es ist die dritte Welle der Pandemie mit der Corona - Pandemie, aber auch wenn die Infektionszahlen steigen. Wir müssen uns angreifen, wenn wir noch nicht so viel Geld verdienen können, sagte er dem Redaktionsnetzwerk Deutschland ( RND ). Es sei wichtig, dass die Pflegebedürftigen in der Pandemie in den vergangenen Jahren immer wieder in die Höhe schnellen, weil sie sich impfen lassen konnten. Die Pflegebedürftigen hatte sich im vergangenen Jahr mit dem Coronavirus angesteckt und damit auch


In [42]:
print(d.iloc[0]['CONTENT'])

Nun ist es also vom Robert-Koch-Institut (RKI) bestätigt: Die dritte Welle hat begonnen, die Infektionszahlen steigen – und auch die 7-Tage-Inzidenz geht wieder nach oben. Leider war das zu erwarten: Bei Lockdown-Lockerungen mit gleichzeitiger Anwesenheit von Virusvarianten, ohne eine gute Öffnungsstrategie und ohne schnellere Impfungen hat der Erreger ein leichtes Spiel. Es ist absehbar, dass nicht nur die Zahl der Infizierten steigen wird, sondern auch die derjenigen, die in Kliniken liegen. Die Impfungen der Höchstbetagten hilft hier nicht viel: Die Corona-Patienten in den Kliniken waren auch in der Vergangenheit häufig unter 80 Jahre alt.Da stimmt eine weitere Nachricht nicht gerade fröhlich: Die deutschen Kranken- und Pflegeheime haben im vergangenen Jahr 0,5 Prozent ihrer Pflegekräfte verloren – und das, obwohl die Bundesagentur für Arbeit und der Gesundheitsminister sich um Anwerbungen im In- und Ausland bemüht hatten. Der Weltbund der Krankenschwestern und -pfleger meldet zudem

In [46]:
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else: return super().find_class(module, name)

with open('./data/sent-cluster/train_set-2.pt', mode='rb') as f:
  train_set = CPU_Unpickler(f).load()

with open('./data/sent-cluster/test_set-2.pt', mode='rb') as f:
  test_set = CPU_Unpickler(f).load()

In [49]:
train_set.dataset.df.iloc[train_set.indices]['clusters'].str.len().sum()

659350

In [48]:
train_set.dataset.df.iloc[train_set.indices]

,clusters,enc_tokens,dec_tokens,y_tokens
6910,[Allerdings liefen zu diesem Zeitpunkt die Prü...,"{'input_ids': [[tensor(3655), tensor(20314), t...","{'input_ids': [[tensor(3), tensor(3655), tenso...","[[tensor(3655), tensor(20314), tensor(81), ten..."
72600,[Die österreichische Regierung berät an diesem...,"{'input_ids': [[tensor(125), tensor(11675), te...","{'input_ids': [[tensor(3), tensor(125), tensor...","[[tensor(125), tensor(11675), tensor(2154), te..."
52485,"[Wichtig sei, überhaupt eine Maske korrekt zu ...","{'input_ids': [[tensor(18840), tensor(350), te...","{'input_ids': [[tensor(3), tensor(18840), tens...","[[tensor(18840), tensor(350), tensor(26918), t..."
103048,[(Foto: picture alliance / ASSOCIATED PRESS)\n...,"{'input_ids': [[tensor(26954), tensor(6463), t...","{'input_ids': [[tensor(3), tensor(26954), tens...","[[tensor(26954), tensor(6463), tensor(26964), ..."
122760,[Zudem bestreitet die DFB-Elf ein Spiel in Rum...,"{'input_ids': [[tensor(2919), tensor(19530), t...","{'input_ids': [[tensor(3), tensor(2919), tenso...","[[tensor(2919), tensor(19530), tensor(30), ten..."
...,...,...,...,...
151493,[An diesem Abend gab es wichtigere Dinge.],"{'input_ids': [[tensor(152), tensor(798), tens...","{'input_ids': [[tensor(3), tensor(152), tensor...","[[tensor(152), tensor(798), tensor(4253), tens..."
140011,[Ansonsten gibt sich die DFB-Elf extrem bieder...,"{'input_ids': [[tensor(19064), tensor(893), te...","{'input_ids': [[tensor(3), tensor(19064), tens...","[[tensor(19064), tensor(893), tensor(144), ten..."
192497,[Eine abschließende Entscheidung zu den Plänen...,"{'input_ids': [[tensor(917), tensor(22605), te...","{'input_ids': [[tensor(3), tensor(917), tensor...","[[tensor(917), tensor(22605), tensor(1410), te..."
162440,[Sie sind 34 und 32.],"{'input_ids': [[tensor(371), tensor(287), tens...","{'input_ids': [[tensor(3), tensor(371), tensor...","[[tensor(371), tensor(287), tensor(4563), tens..."
